In [1]:
from pyspark import SparkConf, SparkContext
import math, sys, glob, random
import sympy
import numpy as np
from nltk import ngrams, word_tokenize
import json
import matplotlib.pyplot as plt
import os
from itertools import *
# from tqdm.notebook import trange, tqdm
plt.style.use('ggplot')

conf = SparkConf().setMaster("local").setAppName("Final")
sc = SparkContext(conf=conf)

In [2]:
%%time
gossip = []
data_path = []
error = 0
success = 0
for root, dirs, files in os.walk("./data/all"):

    for file in files:
        with open(f'{root}/{file}') as f:
            try:
                d = json.load(f)
                d['author']
                d['messages'][0]
                if d['author'] == None:
                    error += 1
                    continue
                gossip.append(d)
                data_path.append(f'{root}/{file}')
                success += 1
            except:
                error += 1
#                 print(f'ERROR: {root}/{file}')



12805
CPU times: user 1min 51s, sys: 26.4 s, total: 2min 17s
Wall time: 3min 14s


In [3]:
print(success)
print(error)

865042
12805


In [3]:
# !cat ./data/all/politics/politics/20011215/M.1008356633.A.json

{"article_id": "M.1008356633.A", "article_title": "從kkcity轉來的  我們嗜血的媒體在哪裡??????? ", "author": "blaketitle (我是大西瓜)", "board": "politics", "content": "作者 [email protected] ( . 看板 politics 標題 [轉錄]北市警察憑什麼搶走我們手中的中華民國國旗？ 時間 興大天樞資訊網 (Thu Dec 13 12:12:13 2001) [離開] [PgUp] [PgDn] 發信人: 仍在流浪中 < [email protected] > 看板: taipei 標 題: 北市警察憑什麼搶走我們手中的中華民國國旗？ 發信站: OpenFind 網路論壇 (Thu Dec 13 06:02:52 2001) 轉信站: Pivotnchunews.nchuccnews.nchuctugatenews.nctufeeder.seed.net.twb Origin: localhost Eric Lo 2001/12/13 AM 02:31:40 北市警察憑什麼搶走我們手中的中華民國國旗？ 十二月十日，在中山足球場，中韓之戰。為了讓我國的足球女將能在國旗的支持下，打贏 這場重要的中韓之戰，我們幾個朋友帶了國旗為中華隊加油。 下午四點，我們打開國旗組合旗竿要開始加油，不多久就發現警察已經近身，並要求我們 將旗幟收起來，由於之前亞洲盃溜冰賽台東模式已有先例，於是我們向警員出示陸委會的 文件。 陸委會林副主委於民國九十年十月五日主持例行記者會紀錄： 主辦國際運動總會正式授權的國際錦標賽應遵守奧會模式 問：昨天在臺東舉行的亞洲杯溜冰賽，中共代表隊抗議我方懸掛國旗，並要求拆除。對於 中共代表隊不合理的要求，陸委會看法如何？ 答： 國內各單項協會等在主辦國際運動總會正式授權的國際錦標賽時，應遵守奧會模式， 但奧會模式僅限於競技場合內，並不適用於一般民眾身上，因此對於民眾在觀眾席 上揮舞國旗或在會場外插國旗等自發行為，若不干擾到比賽之正常進行，主辦單位即無 從勸阻。 我們理解大陸團隊在參加國際或兩岸交流時，必須背負政治干預交流的沈重包袱， 對這些專業人士，我們寄予同情。 相互尊重是從事兩岸交流及參與各種國際競賽起碼應有的風度，

In [4]:
%%time
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(gossip, f, ensure_ascii=False, indent=4)

CPU times: user 8min 9s, sys: 12.5 s, total: 8min 21s
Wall time: 8min 32s


In [5]:
%%time
with open('data_path.json', 'w', encoding='utf-8') as f:
    json.dump(data_path, f, ensure_ascii=False, indent=4)

CPU times: user 772 ms, sys: 100 ms, total: 872 ms
Wall time: 869 ms


In [ ]:
# %%time
# with open('./data.json') as f:
#     gossip = json.load(f)

In [ ]:
gossip[0].keys()

In [ ]:
gossip[0]['messages']

## Graph Construction

In [ ]:
all_user = {}
tag = {'推':1, '噓':-1, '→':0}

### comment : 1：贊, 0：沒情緒, -1：噓
# Mapper : (Author, [(push_user, comment), (push_user2, comment), ...])
def load_users(board):
    result = []
    for artical in board:
        if(artical['author'] not in all_user):
            all_user[artical['author']] = len(all_user)
        
        author = all_user[artical['author']]
        
        pusher_comment = []
        for push_artical in artical['messages']:
            if(push_artical['push_userid'] not in all_user):
                all_user[push_artical['push_userid']] = len(all_user)
            pusher_comment.append((all_user[push_artical['push_userid']], tag[push_artical['push_tag']]))
            
        result.append((author, pusher_comment))
    return result        

In [ ]:
%%time
gossip_graph = sc.parallelize(load_users(gossip))

In [ ]:
gossip_graph.collect()[0]

In [8]:
g = gossip_graph.flatMap(lambda x: [(x[0], (xx[0], xx[1])) for xx in x[1]])

In [9]:
g.collect()[22]

(8, (12, 1))

In [10]:
from operator import add
talk = g.map(lambda x: ((x[0], x[1][0]), x[1][1]))
talk2 = talk.reduceByKey(add)

In [11]:
len(talk.collect())

10731

In [12]:
len(talk2.collect())

6901

In [13]:
len(talk2.filter(lambda x: x[1] < 0).collect())

974